In [1]:
## Importing necessary packages
import configparser
import snowflake.connector
import os
from datetime import date
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization
import pandas as pd
import numpy as np
import copy

In [2]:
## Setting up Snowflake Connection
config = configparser.ConfigParser()

config.read('/tech/appl/default/user/pa08042e/toad.cfg')
snowflake_pass = config['SNOWFLAKE']['secret_passphrase']

with open("/tech/appl/default/user/pa08042e/rsa_snow.p8", "rb") as key:
    private_key= serialization.load_pem_private_key(
        key.read(),
        password= snowflake_pass.encode(),
        backend=default_backend()
    )

pkb = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

ctx = snowflake.connector.connect(
    user='pa08042',
    account='hfsg_prod.us-east-1',
    private_key=pkb,
    database='user_db',
    schema='AD1_PA08042'
    )

In [3]:
start_date, end_date = '\'2025-05-19\'', '\'2025-05-25\''

query = f"""
SELECT POLICY_ID, TO_DATE(CAST(POL_EFF_DT AS VARCHAR),'YYYYMMDD') AS POL_EFF_DT,
STATE_ABBR, ACCT_CR_IND, HH_COMP, INS_SCORE_GRP, INSURED_AGE, 
POL_NEW_RNW_CD, RATE_PLAN_CD, INS_SCORE_CD, YEARS_WITH_HIG, 
CLEAN_DIRTY, POL_LATST_INCID_MO_AGE_GRP, FLAT_CANCEL_FLAG, PREV_TERM_LATEST_PREMIUM,
-- Case Statements
CASE WHEN CASE WHEN sum(FRST_TRANS_PREM) IS NOT NULL THEN sum(FRST_TRANS_PREM) ELSE 0 END = 0 THEN NULL
       ELSE ((CASE WHEN sum(CASE WHEN (LOWER(POL_NEW_RNW_CD) <> 'n'
          AND CASE
            WHEN PREV_TERM_LATEST_PREMIUM IS NOT NULL THEN PREV_TERM_LATEST_PREMIUM
            ELSE 0
          END <> 0
        ) THEN FRST_TRANS_PREM
        ELSE NULL
      END) IS NOT NULL THEN sum(CASE
        WHEN (
          LOWER(POL_NEW_RNW_CD) <> 'n'
          AND CASE
            WHEN PREV_TERM_LATEST_PREMIUM IS NOT NULL THEN PREV_TERM_LATEST_PREMIUM
            ELSE 0
          END <> 0
        ) THEN FRST_TRANS_PREM
        ELSE NULL
      END)
      ELSE 0
    END / NULLIF(CASE
      WHEN sum(CASE
        WHEN (
          LOWER(POL_NEW_RNW_CD) <> 'n'
          AND CASE
            WHEN PREV_TERM_LATEST_PREMIUM IS NOT NULL THEN PREV_TERM_LATEST_PREMIUM
            ELSE 0
          END <> 0
        ) THEN PREV_TERM_LATEST_PREMIUM
        ELSE NULL
      END) IS NOT NULL THEN sum(CASE
        WHEN (
          LOWER(POL_NEW_RNW_CD) <> 'n'
          AND CASE
            WHEN PREV_TERM_LATEST_PREMIUM IS NOT NULL THEN PREV_TERM_LATEST_PREMIUM
            ELSE 0
          END <> 0
        ) THEN PREV_TERM_LATEST_PREMIUM
        ELSE NULL
      END)
      ELSE 0
    END,0.0)) - 1) END AS OFFERED_PREMIUM_CHANGE
FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_PERSISTENCY_NOWCO"
WHERE TO_DATE(CAST(POL_EFF_DT AS varchar),'yyyymmdd') BETWEEN {start_date} AND {end_date}
AND LOWER(POL_NEW_RNW_CD) = 'r'
AND PREV_TERM_LATEST_PREMIUM >= 300 -- Added per Karissa's request
AND (STATE_ABBR IS NULL OR LOWER(STATE_ABBR) IN (
      'ak', 'al', 'ar', 'az', 'co', 'ct', 'dc', 'de', 'fl', 'ga', 'ia', 'id', 'il', 
      'in', 'ks', 'ky', 'la', 'md', 'me', 'mi', 'mn', 'mo', 'ms', 'mt', 'nd', 'ne', 
      'nh', 'nj', 'nm', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 
      'va', 'vt', 'wi', 'wv', 'wy'))
AND (POL_LATST_INCID_MO_AGE_GRP IS NULL OR LOWER(POL_LATST_INCID_MO_AGE_GRP) IN ('0', '13-24', '25-36', '37', '38-60', '61'))
AND FLAT_CANCEL_FLAG = 0
GROUP BY POLICY_ID, POL_EFF_DT, STATE_ABBR, ACCT_CR_IND, 
HH_COMP, INS_SCORE_GRP, INSURED_AGE, POL_NEW_RNW_CD, RATE_PLAN_CD, 
INS_SCORE_CD, YEARS_WITH_HIG, CLEAN_DIRTY, POL_LATST_INCID_MO_AGE_GRP, FLAT_CANCEL_FLAG, PREV_TERM_LATEST_PREMIUM
HAVING OFFERED_PREMIUM_CHANGE >= 0.4
ORDER BY POL_EFF_DT;
"""

In [4]:
# Get data from Snowflake
cs = ctx.cursor()
try:
    cs.execute(query)
    df = cs.fetch_pandas_all()
finally:
    cs.close()
ctx.close()

In [5]:
import openpyxl
formatted_date = date.today().strftime("%m%d%y")
excel_filename = f'NowCo_Auto_Upcoming_Renewals_{formatted_date}.xlsx'
df.to_excel(excel_filename, index=False)
wb = openpyxl.load_workbook(excel_filename)
ws = wb["Sheet1"]

# Iterate through all columns in the worksheet, use the column_dimensions property to get the column object, and set the auto_size attribute to True for automatic column width adjustment
for col in ws.columns:
    ws.column_dimensions[col[0].column_letter].auto_size = True

# Use the save method to save the modified Excel file
wb.save(excel_filename)

In [6]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.multipart import MIMEBase
from email import encoders
from openpyxl import load_workbook

formatted_start = start_date[6:8] + "/" + start_date[9:11] + "/" + start_date[1:5]
formatted_end = end_date[6:8] + "/" + end_date[9:11] + "/" + end_date[1:5]

# Create the root message and fill in the from, to, and subject headers
msg = MIMEMultipart('related')
msg['Subject'] = f'NowCo Auto Upcoming Renewals: {formatted_start} - {formatted_end}'
msg['From'] = 'ProductAnalytics'
msg['To'] = 'peter.alonzo@thehartford.com'
msg['X-MSMail-Priority'] = 'High'

# Create the body with HTML
html = f"""\
<html>
  <body>
    <p>Please see the attached Excel file containing upcoming NowCo Auto Renewals.</p>
    <p>The following filters are being applied:</p>
    <p></p>
    <p>POL_EFF_DT: {formatted_start} to {formatted_end}</p>
    <p>POL_NEW_RNW_CD = R</p>
    <p>STATE_ABBR IN [ak,al,ar,az,co,ct,dc,de,fl,ga,ia,id,il,in,ks,ky,la,md,me,mi,mn,mo,ms,mt,nd,ne,nh,nj,nm,oh,ok,or,pa,ri,sc,sd,tn,tx,ut,va,vt,wi,wv,wy,null]</p>
    <p>OFFERED_PREMIUM_CHANGE >= 0.4</p>
    <p>POL_LATST_INCID_MO_AGE_GRP IN [0,13-24,25-36,37,38-60,61,null]</p>
    <p>FLAT_CANCEL_FLAG = 0</p>
    <p>PREV_TERM_LATEST_PREMIUM >= 300</p>
  </body>
</html>
"""

part1 = MIMEText(html, 'html')
msg.attach(part1)

with open(excel_filename, 'rb') as attachment:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header(
        'Content-Disposition',
        f'attachment; filename= {excel_filename}',
    )
    msg.attach(part)

# Send the message via our own SMTP server, but don't include the envelope header
s = smtplib.SMTP('localhost')
s.sendmail('DoNotReply', ['peter.alonzo@thehartford.com', 'karissa.christiano@thehartford.com'], msg.as_string())
s.quit()
# , 'kevin.bodie@thehartford.com'

(221, b'2.0.0 Bye')